### Welcome to your first casestudy
* In this case study you have to scrape weather data from the website "http://www.estesparkweather.net/archive_reports.php?date=200901"
* Scrape all the available attributes of weather data for each day from 2009-01-01 to 2018-10-28
* Ignore records for missing days
* Represent the scraped data as pandas dataframe object.

### Dataframe specific deatails
* Expected column names (order dose not matter):
```
 ['Average temperature (°F)', 'Average humidity (%)',
 'Average dewpoint (°F)', 'Average barometer (in)',
 'Average windspeed (mph)', 'Average gustspeed (mph)',
 'Average direction (°deg)', 'Rainfall for month (in)',
 'Rainfall for year (in)', 'Maximum rain per minute',
 'Maximum temperature (°F)', 'Minimum temperature (°F)',
 'Maximum humidity (%)', 'Minimum humidity (%)', 'Maximum pressure',
 'Minimum pressure', 'Maximum windspeed (mph)',
 'Maximum gust speed (mph)', 'Maximum heat index (°F)']
```
* Each record in the dataframe corresponds to weather deatils of a given day
* Make sure the index column is date-time format (yyyy-mm-dd)
* Perform necessary data cleaning and type cast each attributes to relevent data type

### Saving the dataframe
* Once you are done with you scrapping save your dataframe as pickle file by name 'dataframe.pk'
### Sample code to save pickle file
```python
import pickle
with open("dataframe.pk", "wb") as file:
    pickle.dump(<your_dataframe>, file)
```

### Run the below cell to import necessary packages
These packages should be sufficient to perform you task
In case if you are looking are any other packages run **!pip3 install --user with in a cell**

In [370]:
import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd
import urllib
import re
import pickle
from datetime import datetime

In [371]:
#### Start you code here, you are free to add any number of cells
url="https://www.estesparkweather.net/archive_reports.php?date="
date_list = pd.date_range(start="2009-01-01", end="2018-10-28", freq="D")
# date_list = pd.date_range(start="2009-01-01", end="2009-01-02", freq="D")

url_list={}

for date in date_list:
    custom_url=url+date.strftime("%Y%m")
    day=date.strftime("%Y %b %d")

    if custom_url in url_list:
        url_list[custom_url].append(day)
    else:
        url_list[custom_url]=[day]
        
print(url_list)

column= ['Average temperature (°F)', 'Average humidity (%)',
 'Average dewpoint (°F)', 'Average barometer (in)',
 'Average windspeed (mph)', 'Average gustspeed (mph)',
 'Average direction (°deg)', 'Rainfall for month (in)',
 'Rainfall for year (in)', 'Maximum rain per minute',
 'Maximum temperature (°F)', 'Minimum temperature (°F)',
 'Maximum humidity (%)', 'Minimum humidity (%)', 'Maximum pressure',
 'Minimum pressure', 'Maximum windspeed (mph)',
 'Maximum gust speed (mph)', 'Maximum heat index (°F)']



{'https://www.estesparkweather.net/archive_reports.php?date=200901': ['2009 Jan 01', '2009 Jan 02', '2009 Jan 03', '2009 Jan 04', '2009 Jan 05', '2009 Jan 06', '2009 Jan 07', '2009 Jan 08', '2009 Jan 09', '2009 Jan 10', '2009 Jan 11', '2009 Jan 12', '2009 Jan 13', '2009 Jan 14', '2009 Jan 15', '2009 Jan 16', '2009 Jan 17', '2009 Jan 18', '2009 Jan 19', '2009 Jan 20', '2009 Jan 21', '2009 Jan 22', '2009 Jan 23', '2009 Jan 24', '2009 Jan 25', '2009 Jan 26', '2009 Jan 27', '2009 Jan 28', '2009 Jan 29', '2009 Jan 30', '2009 Jan 31'], 'https://www.estesparkweather.net/archive_reports.php?date=200902': ['2009 Feb 01', '2009 Feb 02', '2009 Feb 03', '2009 Feb 04', '2009 Feb 05', '2009 Feb 06', '2009 Feb 07', '2009 Feb 08', '2009 Feb 09', '2009 Feb 10', '2009 Feb 11', '2009 Feb 12', '2009 Feb 13', '2009 Feb 14', '2009 Feb 15', '2009 Feb 16', '2009 Feb 17', '2009 Feb 18', '2009 Feb 19', '2009 Feb 20', '2009 Feb 21', '2009 Feb 22', '2009 Feb 23', '2009 Feb 24', '2009 Feb 25', '2009 Feb 26', '2009

In [372]:
def get_url_response(url):
    response = requests.get(url)
    if response.status_code != 200:
        print("failed to hit url: status_code=",response.status_code,"url",url)
        
    soup = BeautifulSoup(response.content, "html.parser")
    return soup

In [373]:
day_temp=[]
index=[]
for url, all_date in url_list.items():
    # print(url)
    soup = get_url_response(url)
    date=[i[5:8].replace('-',' ')+" "+str(int(i[-2:])) for i in all_date]
    tables=soup.find_all(name="table", attrs={"cellpadding":"3", "cellspacing":"0", "border":"0"})
    
    for i in range(len(tables)):

        td=tables[i].findAll("td")
        month_day=td[0].text[0:6].strip()

        if month_day in date:
            row_index=datetime.strptime(all_date[0][0:4]+' '+month_day, "%Y %b %d").strftime("%Y-%m-%d")
            index.append(row_index)

            # print(month_day)
            day_temp.append({'Date':row_index
            ,'Average temperature (°F)':int(re.findall(r'\d+',td[2].text)[0])
            , 'Average humidity (%)':int(re.findall(r'\d+',td[4].text)[0])
            , 'Average dewpoint (°F)':int(re.findall(r'\d+',td[6].text)[0])
            , 'Average barometer (in)':int(re.findall(r'\d+',td[8].text)[0])
            ,'Average windspeed (mph)':float(re.findall(r'\d+.\d+',td[10].text)[0])
            , 'Average gustspeed (mph)':int(re.findall(r'\d+',td[12].text)[0])
            ,'Average direction (°deg)':int(re.findall(r'\d+',td[14].text)[0])
            , 'Rainfall for month (in)':int(re.findall(r'\d+',td[16].text)[0])
            ,'Rainfall for year (in)':int(re.findall(r'\d+',td[18].text)[0])
            , 'Maximum rain per minute':int(re.findall(r'\d+',td[20].text)[0])
            ,'Maximum temperature (°F)':float(re.findall(r'\d+.\d+',td[22].text)[0])
            , 'Minimum temperature (°F)':int(re.findall(r'\d+',td[24].text)[0])
            ,'Maximum humidity (%)':int(re.findall(r'\d+',td[26].text)[0])
            , 'Minimum humidity (%)':int(re.findall(r'\d+',td[28].text)[0])
            , 'Maximum pressure':float(re.findall(r'\d+.\d+',td[30].text)[0])
            ,'Minimum pressure':float(re.findall(r'\d+.\d+',td[32].text)[0])
            , 'Maximum windspeed (mph)':int(re.findall(r'\d+',td[34].text)[0])
            ,'Maximum gust speed (mph)':int(re.findall(r'\d+',td[36].text)[0])
            , 'Maximum heat index (°F)':int(re.findall(r'\d+',td[38].text)[0])})
df=pd.DataFrame(day_temp, index=index)
df['Date']=pd.to_datetime(df['Date'])
df.set_index('Date',inplace=True)
df.head(5)
    

,Average temperature (°F),Average humidity (%),Average dewpoint (°F),Average barometer (in),Average windspeed (mph),Average gustspeed (mph),Average direction (°deg),Rainfall for month (in),Rainfall for year (in),Maximum rain per minute,Maximum temperature (°F),Minimum temperature (°F),Maximum humidity (%),Minimum humidity (%),Maximum pressure,Minimum pressure,Maximum windspeed (mph),Maximum gust speed (mph),Maximum heat index (°F)
Date,,,,,,,,,,,,,,,,,,,
2009-01-01,37,35,12,29,26.4,36,274,0,0,0,40.1,34,44,27,29.762,29.596,41,59,40
2009-01-02,43,32,14,29,12.8,18,240,0,0,0,52.8,37,43,16,29.669,29.268,35,51,52
2009-01-03,25,60,12,29,8.3,12,290,0,0,0,41.2,6,89,35,30.232,29.260,25,38,41
2009-01-04,9,67,0,30,2.9,4,47,0,0,0,19.4,0,79,35,30.566,30.227,12,20,32
2009-01-05,23,30,5,29,16.7,23,265,0,0,0,30.3,15,56,13,30.233,29.568,38,53,32


In [377]:
# with open("dataframe.pk", "wb") as file:
#     pickle.dump(df, file)

In [378]:
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype

assert is_datetime64_dtype(df.index)

for i in df.columns:
    assert is_numeric_dtype(df[i])

In [379]:
import numpy as np

mean = round(np.mean(df["2011-08-1":"2011-08-20"]["Average windspeed (mph)"]), 2)
assert str(mean) == '4.64'

std2 = round(np.std(df["2011-04-20":"2012-01-1"]["Maximum pressure"]), 2)
assert str(std2) == '0.27'

max = round(np.max(df["2011-04-20":"2012-01-1"]["Maximum temperature (°F)"]), 2)
assert str(max) == '89.7'